In [0]:
!pip install opencc-python-reimplemented # 安裝 OpenCC
!wget https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2



from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2', dictionary={})
next(iter(wiki_corpus.get_texts()))[:10]

text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

from gensim.models import word2vec


# 設定
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

#sg：演算法，預設為0，代表是CBOW，若設為1則是使用Skip-Gra
#size：訓練出的詞向量會有幾維
#window：代表input word 與預測word的最大距離，若設為1，代表向左向右各看一個詞
#min_count：min_count最少出現幾次，才能當作是訓練資料
#workers:執行緒數目
#negative：大於0，代表採用negative sampling，設置多少個noise words
#epoch：1個epoch表示過了1遍訓練集中的所有樣本
#batch-size：1次迭代所使用的樣本量

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')


from gensim.models import word2vec

string = '微生物'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

     |████████████████████████████████| 491kB 2.8MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.5-py2.py3-none-any.whl size=485664 sha256=2c9e84cc76978421110ed0940c119a55f9a734d62c708e97f06ec275d3d381de
  Stored in directory: /root/.cache/pip/wheels/36/a0/10/888b9ac7f10154caaa6a73270b1f085e0a7b241baa672bbe49
Successfully built opencc-python-reimplemented
--2020-04-23 01:27:22--  https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172586252 (165M) [application/octet-stream]
Saving to: ‘zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2’

zhwiki-20200301-pag 100%[===================>] 164.59M  4.65MB/s    in 40s     

2020-04-23 01:28:03 (4.1

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


微生物
('細菌', 0.8878570795059204)
('菌', 0.8606401681900024)
('代謝', 0.8552396893501282)
('病理', 0.8490051627159119)
('毒素', 0.8462592363357544)
('致病', 0.8448585271835327)
('免疫', 0.8435909748077393)
('藻類', 0.8424044251441956)
('真菌', 0.8414219617843628)
('激素', 0.8384472727775574)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
